In [16]:
# %pip install langchain-pinecone
# %pip install pypdf
# %pip install sentence-transformers
# %pip install ollama
# %pip install -U "autogen-ext[ollama]"
# %pip install crewai
# %pip install -U protobuf


In [48]:
import os
import langchain
from langchain_community.document_loaders import PyPDFLoader

from pinecone import Pinecone
from pinecone import ServerlessSpec

from sentence_transformers import SentenceTransformer

import ollama
from crewai import Agent, Crew, Task
from crewai import LLM
from crewai.tools import tool

from ollama import chat
from ollama import ChatResponse

from crewai.flow.flow import Flow, listen, start
import asyncio

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_text_splitters import MarkdownHeaderTextSplitter

In [ ]:

# texts = text_splitter.split_text(document)

In [ ]:
pinecone_api_key  = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)

index_name = "logistics-faq"

pc.delete_index(index_name)
pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

index = pc.Index(index_name)

In [54]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')

In [55]:
documents_path = os.path.join(os.getcwd(),"documents")
print(documents_path)

c:\Users\Anoopkumarjain\Documents\git_reps\GenAI\documents


In [56]:
def store_embeddings():
    idx = 1
    for file in os.listdir("documents"):
        f = open(os.path.join(documents_path, file))
        text = f.read()
        # print(text)
        text_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=[("--","Pair")])
        splits = text_splitter.split_text(text)
        print(len(splits))
        embeddings =  model.encode([text]) 
        vectors = [{"id":str(idx),"values":embeddings[0],"metadata": {'text': text}}]
        index.upsert(vectors=vectors,namespace="douments")
        idx+=1

In [57]:
store_embeddings()

12


In [58]:
def initialise_ollama(model_name="tinyllama:1.1b"):
    ollama.pull(model_name)
    print(ollama.list())
# initialise_ollama()

In [59]:
llm = LLM(
    model="ollama/tinyllama:1.1b",
    base_url="http://localhost:11434"
)




In [60]:
query = "How long will it take for my package to arrive?"

In [61]:

class ExampleFlow(Flow):
    model = llm
    @start()
    def get_content(self)->list:
        """Useful to generate answers from retrived chunks of documents"""
        query_embedding = model.encode(query).tolist()

        print("===========Retrieving content===========")

        results = index.query(
            namespace="douments",
            vector=query_embedding,
            top_k=3,
            include_values=False,
            include_metadata=True
        )

        return [match["metadata"]["text"].replace("\t"," ").replace("\n"," ") for match in results["matches"]]

    @listen(get_content)
    def get_response(self, get_content):
        response: ChatResponse = chat(model="tinyllama:1.1b", messages=[
        {
            'role': 'user',
            'content': f"Answer user query from provided content: \n query:{query}, content: {get_content}, Answer:",
        },
        ])
        # or access fields directly from the response object
        return response.message.content



In [62]:
flow = ExampleFlow()
# flow.plot()
result = await flow.kickoff_async()

print(f"Generated fun fact: {result}")

╭──────────────────────────────────────────────── Flow Execution ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Starting Flow Execution                                                                                        │
│  Name: ExampleFlow                                                                                              │
│  ID: cf7275cd-852e-4329-941f-e62c93b8531d                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

 Flow started with ID: cf7275cd-852e-4329-941f-e62c93b8531d

c:\Users\Anoopkumarjain\anaconda3\envs\kaggle\Lib\site-packages\rich\live.py:231: UserWarning: install "ipywidgets"
for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

===========Retrieving content===========

╭──────────────────────────────────────────────── Flow Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Flow Execution Completed                                                                                       │
│  Name: ExampleFlow                                                                                              │
│  ID: cf7275cd-852e-4329-941f-e62c93b8531d                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Generated fun fact: As per our current shipping policy, your parcel should be delivered within approximately 2-5 business days after the order has been processed and confirmed by our team. However, in case of any delays due to customs clearance or postal issues, it may take a bit longer than this duration. We strive to ensure that our customers receive their orders as soon as possible, and we do our best to maintain our promised delivery time frame. Please be assured that we are working hard to improve the delivery process as well.
